In [ ]:
import pandas as pd
import numpy as np

data_path = 'bladder2.csv'

df = pd.read_csv(data_path)
df = df.iloc[ :,2:]
print(df.head())

In [ ]:
import torch  
from torch.utils.data import DataLoader, TensorDataset  
from sklearn.preprocessing import StandardScaler  

Y_df = df['event']
X_df = df.drop(columns=['event']) 

X_df['rx_number'] = X_df['rx'] * 0.5 + X_df['number'] * 0.5 

X_data_array = X_df.values
Y_data_array = Y_df.values

# 归一化
scaler = StandardScaler() 
X_data_array = scaler.fit_transform(X_data_array)  

 
feature_tensor = torch.tensor(X_data_array, dtype=torch.float32)  
targets_tensor = torch.tensor(Y_data_array, dtype=torch.float32)  
  
batch_size = 8 
dataset = TensorDataset(feature_tensor, targets_tensor)   
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  


In [ ]:
import torch
import torch.nn as nn

class RecurrencePredictor(nn.Module):
    def __init__(self):
        super(RecurrencePredictor, self).__init__()
        self.fc1 = nn.Linear(7, 256)
        self.bn1 = nn.BatchNorm1d(256)  # 添加批量归一化层
        self.fc2 = nn.Linear(256, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc5 = nn.Linear(64, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.fc6 = nn.Linear(32, 1)
        self.relu = nn.PReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, is_feature=False):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.relu(self.bn4(self.fc4(x)))
        x = self.relu(self.bn5(self.fc5(x)))
        if not is_feature:
            x = self.sigmoid(self.fc6(x))
        return x


model = RecurrencePredictor()

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.001)


num_epochs = 50
losses = []
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(loss)
    losses.append(running_loss / len(dataloader))

model.eval()
with torch.no_grad():
    x_pred = []
    for inputs, _ in dataloader:
        extract_x = model(inputs, is_feature=True)
        print(extract_x)
